<a href="https://colab.research.google.com/github/umSamWeiHong/DepressionModel/blob/master/DepressionIndicator_Wei_Hong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some random text


In [225]:
# !pip install tensorflow-gpu

In [226]:
# !nvidia-smi

In [227]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [228]:
import tensorflow as tf
import numpy as np
import pandas as pd
tf.__version__

'2.8.2'

In [229]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning Group Project/depression.csv')
target_name = 'DASS_depression_severity'
y = df[target_name]
df

,Code,Age_Numerical,Gender,Marital,Marital_problems,Religion_binary,Religion_help,Med_or_not,Expenses,Monthly_expenses,...,DASS_stress_numerical,MSPSS_family,MSPSS_friends,MSPSS_significant_others,DASS_depression_binary,DASS_depression_severity,DASS_anxiety_binary,DASS_anxiety_severity,DASS_stress_binary,DASS_stress_severity
0,1,34,0,0,0,1,1,1,2,1,...,0,27.2,24.0,26.0,0,0,0,0,0,0
1,2,32,1,0,0,1,1,1,1,0,...,2,28.0,28.0,28.0,0,0,0,0,0,0
2,3,33,0,0,0,1,1,1,1,0,...,0,28.0,28.0,28.0,0,0,0,0,0,0
3,4,33,1,0,1,1,1,1,2,1,...,18,25.2,21.2,24.0,1,2,1,4,1,1
4,5,39,1,0,0,1,1,1,2,1,...,2,26.0,20.0,28.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,312,23,1,1,0,0,1,1,2,1,...,8,25.2,25.2,27.2,0,0,0,0,0,0
312,313,36,0,1,0,0,0,0,2,1,...,0,23.2,20.0,24.0,0,0,0,0,0,0
313,314,33,0,0,0,0,0,1,2,1,...,0,25.2,19.2,27.2,0,0,0,0,0,0
314,315,25,1,1,0,1,0,1,0,0,...,26,14.0,13.2,13.2,1,4,1,2,1,2


Remove unnecessary columns (features):

In [230]:
print(df.columns)
df.drop(columns=['Code','Residence_high_COVID1','Living_with','Monthly_expenses',
                 'DASS_depression_numerical','DASS_anxiety_numerical','DASS_stress_numerical',
                 'DASS_depression_binary','DASS_depression_severity', 'DASS_anxiety_binary','DASS_anxiety_severity','DASS_stress_binary','DASS_stress_severity',
                #  'Religion_binary',
                 ],
        inplace=True, axis=1)
print(df.columns)

Index(['Code', 'Age_Numerical', 'Gender', 'Marital', 'Marital_problems',
       'Religion_binary', 'Religion_help', 'Med_or_not', 'Expenses',
       'Monthly_expenses', 'Living_with1', 'Living_with', 'Worry_family',
       'Routine_loss', 'Class_hours', 'Stress_class_disrupted',
       'Medical_illness', 'Mental_illness', 'COVID_symptoms',
       'Residence_high_COVID1', 'Residence_high_COVID', 'Quarantine',
       'DASS_depression_numerical', 'DASS_anxiety_numerical',
       'DASS_stress_numerical', 'MSPSS_family', 'MSPSS_friends',
       'MSPSS_significant_others', 'DASS_depression_binary',
       'DASS_depression_severity', 'DASS_anxiety_binary',
       'DASS_anxiety_severity', 'DASS_stress_binary', 'DASS_stress_severity'],
      dtype='object')
Index(['Age_Numerical', 'Gender', 'Marital', 'Marital_problems',
       'Religion_binary', 'Religion_help', 'Med_or_not', 'Expenses',
       'Living_with1', 'Worry_family', 'Routine_loss', 'Class_hours',
       'Stress_class_disrupted', 'Med

One hot encoding for categorical data

In [231]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
df[['Alone','Friends/Coursemates','Family']] = ohe.fit_transform(df['Living_with1'].values.reshape(-1,1))
df[['< RM1000','RM1000 - 3000','> RM3000']] = ohe.fit_transform(df['Expenses'].values.reshape(-1,1))
df = df.drop(columns=['Living_with1','Expenses'])
X = df

Split the dataset into the training set and test set

In [232]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(221, 25) (221,)
(95, 25) (95,)


Call the method below for feature normalisation (for numerical values):

In [240]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

def featureNormalisation():
  numeric_data = list(X.columns)
  categorical_data = ['Gender','Marital','Marital_problems','Religion_binary','Religion_help','Med_or_not','Worry_family','Routine_loss',
                      'Stress_class_disrupted','Medical_illness','Mental_illness','COVID_symptoms','Residence_high_COVID','Quarantine',
                      'Alone','Friends/Coursemates','Family','< RM1000','RM1000 - 3000','> RM3000']
  for category in categorical_data:
    numeric_data.remove(category)

  ct = ColumnTransformer([('scaler', StandardScaler(), numeric_data)], remainder='passthrough')

  X_train_scaled = ct.fit_transform(X_train)
  # print(pd.DataFrame(X_train_scaled, columns = ct.get_feature_names_out()))
  # the test set uses the fitted scaler in train dataset to transform in the test set
  X_test_scaled = ct.transform(X_test)
  return X_train_scaled, X_test_scaled, ct.get_feature_names_out()

Testing for feature normalisation:

In [241]:
print(X_train[0:6], X_test[0:6], sep='\n')
X_train_scaled, X_test_scaled, features = featureNormalisation()

def printDataFrame(df):
  print(pd.DataFrame(df, columns = features))
printDataFrame(X_train_scaled[0:6])
printDataFrame(X_test_scaled[0:6])

     Age_Numerical  Gender  Marital  Marital_problems  Religion_binary  \
287             34       1        0                 0                0   
238             20       0        1                 0                0   
119             35       1        0                 0                1   
28              34       1        0                 0                1   
63              24       0        1                 0                0   
147             20       1        1                 0                1   

     Religion_help  Med_or_not  Worry_family  Routine_loss  Class_hours  ...  \
287              0           1             0             1          3.0  ...   
238              0           1             0             1          8.0  ...   
119              1           1             1             0          2.0  ...   
28               1           1             0             0          5.0  ...   
63               0           1             0             1         10.0  ...   
1

Below is the LDA part, in this case, LDA is used for dimensionality reduction, for the classification, we will use transformer architecture.

In [91]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
def lda():
  lda = LDA(n_components=3)
  x_train_lda = lda.fit_transform(X_train, y_train)
  x_test_lda = lda.transform(X_test)
  lda.explained_variance_ratio_

In [92]:
#lda.explained_variance_ratio_